In [16]:
import numpy as np

# Define state 1, 2, 3 ,4 ,5... 19 as normal state with one-hot encoding
# state 0 and state 20 share the same zero feature vectors.

def feature_map(state):
    zero_model = [0]*19
    zero_model[state-1] = 1
    zero_model = np.array(zero_model)
    zero_model.resize((19,1))
    return np.array(zero_model)

# create a hash table to quickly draw features
feature_hash = {0: np.zeros((19,1)),
                20: np.zeros((19,1))}
for state in range(1,20):
    feature_hash[state] = feature_map(state)


In [21]:
history = []
for episode in range(100):
    local = [10]
    state = 10
    while True:
        if np.random.random() > 0.5:
            state += 1
        else:
            state -= 1
        local.append(state)
        if state == 0 or state == 20:
            history.append(local)
            break

In [19]:
# hand pick the hyper parameters
alpha = 0.4
gamma = 0.8
_lambda = 0.9

# set all ones as initialization
w_last_episode = np.ones((19,1))
w= {}
for hist in history:
    for h in range(1, 100000):
        for t in range(h):
            w[(h,0)] = w_last_episode
            w[(h,t)] = w[(h,t-1)] + alpha*[G(0,2,w[(h,t-1)],hist) - w[(h,t-1)].T@feature_hash[S]]


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])